In [2]:
import pandas as pd
import glob

In [17]:
dfs = []
resfiles = [
    # *glob.glob("../100k_1/2023*.log"),
    # *glob.glob("../1M/2023*.log"),
    # *glob.glob("../500k_1/2023*.log"),
    *glob.glob("../proposal_extra_features/hamlet/*_results.log")
]
for f in resfiles:
    print(f"reading file {f}")
    tdf = pd.read_json(f, lines=True)
    if f == "gpu/2023-06-17_07-08-00_251139_results.log":
        tdf = tdf[tdf.operator != 'KMeans']

    dfs.append(tdf)

df:pd.DataFrame = pd.concat(dfs)
df = df[~df.operator.apply(lambda x: "fail" in x)]
df.to_parquet("sigmod_extra_features_hamlet.parquet")
display(df.head())
print(len(df))

reading file ../proposal_extra_features/hamlet/2023-07-29_11-58-18_331042_results.log


,dataset,join,operator,model,tuple_ratio,feature_ratio,cardinality_T,cardinality_S,complexity,selectivity,times,num_cores,data_characteristics,profile_times,times_std,times_mean
0,book,inner,Noop,materialized,0.557832,1.914246,253120,77848,NaN,0.307554,[1.443004293832928e-06],32,"{'r_S': [27876, 49972], 'c_S': [28022, 53641],...",NaN,-1,1.443004e-06
1,book,inner,Noop,factorized,0.557832,1.914246,253120,77848,NaN,0.307554,[4.609973984770477e-07],32,"{'r_S': [27876, 49972], 'c_S': [28022, 53641],...",NaN,-1,4.609974e-07
2,book,inner,Left multiply,materialized,0.557832,1.914246,253120,77848,2024960.0,0.307554,[0.00210807500116],32,"{'r_S': [27876, 49972], 'c_S': [28022, 53641],...",NaN,-1,2.108075e-03
3,book,inner,Left multiply,factorized,0.557832,1.914246,253120,77848,333488.0,0.307554,[0.000482290000945],32,"{'r_S': [27876, 49972], 'c_S': [28022, 53641],...",NaN,-1,4.822900e-04
4,book,inner,Right multiply,materialized,0.557832,1.914246,253120,77848,2024960.0,0.307554,[0.001969867997104],32,"{'r_S': [27876, 49972], 'c_S': [28022, 53641],...",NaN,-1,1.969868e-03


756


In [13]:
import numpy as np
df.feature_ratio.max()

10.0

In [17]:
df.selectivity.max()

2.0

In [15]:
dfs = []
resfiles = [
    *glob.glob("../100k_1/features*.log"),
    *glob.glob("../1M/features*.log"),
    *glob.glob("../500k_1/features*.log")
]
for f in resfiles:
    print(f"reading file {f}")
    tdf = pd.read_json(f, lines=True)
    if f == "gpu/2023-06-17_07-08-00_251139_results.log":
        tdf = tdf[tdf.operator != 'KMeans']

    dfs.append(tdf)

df:pd.DataFrame = pd.concat(dfs)
df = df.rename(columns={'join':'operator'})
df.drop_duplicates(subset=['dataset', 'operator',], keep="first", inplace=True)
df.to_parquet("sigmod_merged_features.parquet")
display(df.head())
print(len(df))

reading file ../100k_1/features_2023-07-10_12-42-21_849080.log
reading file ../100k_1/features_2023-07-10_12-42-21_830155.log
reading file ../1M/features_2023-07-07_15-27-57_510420.log
reading file ../1M/features_2023-07-07_15-27-57_442052.log
reading file ../1M/features_2023-07-04_05-25-22_210638.log
reading file ../1M/features_2023-07-04_05-25-22_130888.log
reading file ../1M/features_2023-07-08_20-44-58_314495.log
reading file ../1M/features_2023-07-08_20-44-58_033822.log
reading file ../1M/features_2023-07-08_20-44-57_854846.log
reading file ../1M/features_2023-07-07_15-27-57_394280.log
reading file ../1M/features_2023-07-08_20-44-58_038719.log
reading file ../1M/features_2023-07-08_20-44-57_837702.log
reading file ../1M/features_2023-07-05_16-20-01_834834.log
reading file ../1M/features_2023-07-07_15-27-57_421450.log
reading file ../1M/features_2023-07-08_20-44-57_934833.log
reading file ../1M/features_2023-07-08_20-44-58_034324.log
reading file ../1M/features_2023-07-05_16-20-01_

,dataset,operator,features,r_S,c_S,nnz_S,r_T,c_T,nnz_T,TR,FR
0,/user/data/generated/n_R=1-r_T=100000-c_T=10-r...,Noop,"[None, None]","[100000, 526]","[1, 10]","[100000, 4734]",100000,10,1000000,190.114068,10.0
1,/user/data/generated/n_R=1-r_T=100000-c_T=10-r...,Left multiply,"[1000000, 104734, 8000000, 8000000, 837872, 83...","[100000, 526]","[1, 10]","[100000, 4734]",100000,10,1000000,190.114068,10.0
2,/user/data/generated/n_R=1-r_T=100000-c_T=10-r...,Right multiply,"[1000000, 104734, 8000000, 8000000, 837872, 83...","[100000, 526]","[1, 10]","[100000, 4734]",100000,10,1000000,190.114068,10.0
3,/user/data/generated/n_R=1-r_T=100000-c_T=10-r...,Row summation,"[1000000, 104734, 8000000, 8000000, 837872, 83...","[100000, 526]","[1, 10]","[100000, 4734]",100000,10,1000000,190.114068,10.0
4,/user/data/generated/n_R=1-r_T=100000-c_T=10-r...,Column summation,"[1000000, 104734, 8000000, 8000000, 837872, 83...","[100000, 526]","[1, 10]","[100000, 4734]",100000,10,1000000,190.114068,10.0


32130


In [8]:
dfs = []
resfiles = [
    *glob.glob("../hamlet/2023*_results.log")

]
for f in resfiles:
    print(f"reading file {f}")
    tdf = pd.read_json(f, lines=True)
    if f == "gpu/2023-06-17_07-08-00_251139_results.log":
        tdf = tdf[tdf.operator != 'KMeans']

    dfs.append(tdf)

df:pd.DataFrame = pd.concat(dfs)
df = df[~df.operator.apply(lambda x: "fail" in x)]
df.to_parquet("sigmod_merged_hamlet.parquet")
display(df.head())
print(len(df))

reading file ../hamlet/2023-07-11_11-05-34_469967_results.log
reading file ../hamlet/2023-07-11_09-56-55_296381_results.log
reading file ../hamlet/2023-07-11_09-56-55_018046_results.log
reading file ../hamlet/2023-07-11_09-56-54_731834_results.log


,dataset,join,operator,model,tuple_ratio,feature_ratio,cardinality_T,cardinality_S,complexity,selectivity,times,num_cores,data_characteristics,profile_times,times_std,times_mean
0,book,inner,Noop,materialized,0.557832,1.914246,253120,77848,NaN,0.307554,"[1.733002136461436e-06, 4.4099579099565744e-07...",32,NaN,NaN,4.394043e-07,4.886984e-07
1,book,inner,Noop,factorized,0.557832,1.914246,253120,77848,NaN,0.307554,"[7.709968485869467e-07, 3.710010787472129e-07,...",32,NaN,NaN,1.323216e-07,4.107991e-07
2,book,inner,Left multiply,materialized,0.557832,1.914246,253120,77848,2024960.0,0.307554,"[0.001698968997516, 0.001619219998246, 0.00153...",32,NaN,NaN,1.142788e-04,1.574954e-03
3,book,inner,Left multiply,factorized,0.557832,1.914246,253120,77848,333488.0,0.307554,"[0.0008261559996750001, 0.0007124640032990001,...",32,NaN,NaN,2.292410e-04,4.571054e-04
4,book,inner,Right multiply,materialized,0.557832,1.914246,253120,77848,2024960.0,0.307554,"[0.001786032000381, 0.0016204319981620002, 0.0...",32,NaN,NaN,5.726379e-03,3.739731e-03


3024


In [5]:
import os
os.listdir("../hamlet")

['features_2023-07-11_09-56-55_302651.log',
 'features_2023-07-11_09-56-55_024194.log',
 'features_2023-07-11_11-05-34_480017.log',
 'features_2023-07-11_09-56-54_744298.log',
 '2023-07-11_11-05-34_469967_results.log',
 '2023-07-11_09-56-55_296381_results.log',
 '2023-07-11_09-56-55_018046_results.log',
 '2023-07-11_09-56-54_731834_results.log']

In [13]:
dfs = []
resfiles = [
    *glob.glob("../hamlet/features_*.log")
]
for f in resfiles:
    print(f"reading file {f}")
    tdf = pd.read_json(f, lines=True)
    if f == "gpu/2023-06-17_07-08-00_251139_results.log":
        tdf = tdf[tdf.operator != 'KMeans']

    dfs.append(tdf)

df:pd.DataFrame = pd.concat(dfs)
df = df.rename(columns={'join':'operator'})
df.drop_duplicates(subset=['dataset', 'operator',], keep="first", inplace=True)
df.to_parquet("sigmod_merged_hamlet_features.parquet")
display(df.head())
print(len(df))

reading file ../hamlet/features_2023-07-11_09-56-55_302651.log
reading file ../hamlet/features_2023-07-11_09-56-55_024194.log
reading file ../hamlet/features_2023-07-11_11-05-34_480017.log
reading file ../hamlet/features_2023-07-11_09-56-54_744298.log


,dataset,operator,features,r_S,c_S,nnz_S,r_T,c_T,nnz_T,TR,FR
0,book,Noop,"[None, None]","[27876, 49972]","[28022, 53641]","[83628, 249860]",253120,81663,2024960,0.557832,1.914246
1,book,Left multiply,"[2024960, 333488, 16199680, 16199680, 2667904,...","[27876, 49972]","[28022, 53641]","[83628, 249860]",253120,81663,2024960,0.557832,1.914246
2,book,Right multiply,"[2024960, 333488, 16199680, 16199680, 2667904,...","[27876, 49972]","[28022, 53641]","[83628, 249860]",253120,81663,2024960,0.557832,1.914246
3,book,Row summation,"[2024960, 333488, 16199680, 16199680, 2667904,...","[27876, 49972]","[28022, 53641]","[83628, 249860]",253120,81663,2024960,0.557832,1.914246
4,book,Column summation,"[2024960, 333488, 16199680, 16199680, 2667904,...","[27876, 49972]","[28022, 53641]","[83628, 249860]",253120,81663,2024960,0.557832,1.914246


126


In [9]:
df['dataset_size'] = df.dataset.apply(lambda s: s.split('=')[2].split('-')[0]).astype(int)

IndexError: list index out of range

In [10]:
df.groupby(['dataset_size']).num_cores.value_counts()

KeyError: 'dataset_size'

In [11]:
# Some checks
print(df.groupby('operator').count())

                     dataset  join  model  tuple_ratio  feature_ratio  \
operator                                                                
Column summation         168   168    168          168            168   
Column summation T       168   168    168          168            168   
Gaussian                 168   168    168          168            168   
KMeans                   168   168    168          168            168   
LMM                      168   168    168          168            168   
LMM T                    168   168    168          168            168   
Left multiply            168   168    168          168            168   
Left multiply T          168   168    168          168            168   
Linear Regression        168   168    168          168            168   
Logistic Regression      168   168    168          168            168   
Materialization          168   168    168          168            168   
Noop                     168   168    168          

In [41]:
df[df.num_cores == 32].groupby(["operator"]).count()

,dataset,join,model,tuple_ratio,feature_ratio,cardinality_T,cardinality_S,complexity,selectivity,times,num_cores,data_characteristics,profile_times,times_std,times_mean,dataset_size
operator,,,,,,,,,,,,,,,,
Column summation,3570,3570,3570,3570,3570,3570,3570,3570,3570,3570,3570,3570,0,3570,3570,3570
Column summation T,3570,3570,3570,3570,3570,3570,3570,3570,3570,3570,3570,3570,0,3570,3570,3570
Gaussian,3570,3570,3570,3570,3570,3570,3570,3570,3570,3570,3570,3570,0,3570,3570,3570
KMeans,3570,3570,3570,3570,3570,3570,3570,3570,3570,3570,3570,3570,0,3570,3570,3570
LMM,3570,3570,3570,3570,3570,3570,3570,3570,3570,3570,3570,3570,0,3570,3570,3570
LMM T,3570,3570,3570,3570,3570,3570,3570,3570,3570,3570,3570,3570,0,3570,3570,3570
Left multiply,3570,3570,3570,3570,3570,3570,3570,3570,3570,3570,3570,3570,0,3570,3570,3570
Left multiply T,3570,3570,3570,3570,3570,3570,3570,3570,3570,3570,3570,3570,0,3570,3570,3570
Linear Regression,3570,3570,3570,3570,3570,3570,3570,3570,3570,3570,3570,3570,0,3570,3570,3570


In [14]:
df.num_cores = df.num_cores.astype(str)
